[ CNN Fashion MNIST MODEL ]

<1> <hr>

In [120]:
# [1-1] 모듈 로딩 및 데이터 준비
import torch                                   # 텐서 및 기본 함수들 관련 모듈
import torch.nn as nn                          # 인공신경망 관련 모듈
 
from torchvision.datasets import FashionMNIST  # 비전 관련 내장 데이터셋 모듈
from torch.utils.data import DataLoader        # pytorch의 데이터셋 관련 모듈

import torchvision.transforms as transforms    # 비전 관련 이미지 증강/변환 관련 모듈 

import matplotlib.pyplot as plt                # 이미지 시각화 

import torch.nn.functional as F

In [121]:
# [1-2] 이미지 관련 준비
IMG_ROOT = '../D0402/'   # 이미지 데이터 저장 폴더 경로 

In [122]:
# [1-3] 데이터 로딩

# train
trainDS = FashionMNIST(root=IMG_ROOT, 
                       download=True,
                       train=True,
                       transform=transforms.ToTensor())

In [144]:
print(trainDS)

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: ../D0402/
    Split: Train
    StandardTransform
Transform: ToTensor()


In [123]:
# test
testDS = FashionMNIST(root=IMG_ROOT, 
                       download=True,
                       train=False,
                       transform=transforms.ToTensor())

In [124]:
# 데이터셋 체크
# - 타입
print(f'타입 : {type(testDS)}')

# - 속성 : 클래스 정보
print(f'클래스 : {testDS.classes}')
print(f'클래스 인덱스 : {testDS.class_to_idx}')

타입 : <class 'torchvision.datasets.mnist.FashionMNIST'>
클래스 : ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
클래스 인덱스 : {'T-shirt/top': 0, 'Trouser': 1, 'Pullover': 2, 'Dress': 3, 'Coat': 4, 'Sandal': 5, 'Shirt': 6, 'Sneaker': 7, 'Bag': 8, 'Ankle boot': 9}


In [125]:
# - 속성 : 데이터와 타겟
print(f'targets : {testDS.targets}')
print(f'data : {testDS.data.shape}')

targets : tensor([9, 2, 1,  ..., 8, 1, 5])
data : torch.Size([10000, 28, 28])


In [126]:
# 이미지 데이터와 타겟 확인
idx_to_classes = {v:k for k,v in testDS.class_to_idx.items()}

plt.imshow(testDS.data[0], cmap = 'gray')
plt.title(idx_to_classes[testDS.targets[0].item()])
plt.show()

In [127]:
# 로우 데이터 확인
print(f'raw : {testDS.data[0].min()} {testDS.data[0].max()}')

raw : 0 255


<2> <hr>

<3> <hr>

In [128]:
# 학습 관련 설정
EPOCHS = 10
BATCH_SIZE = 100
T_ITERATION = int(trainDS.data.shape[0] / BATCH_SIZE)
LR = 0.001
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f'DEVICE = {DEVICE}')

DEVICE = cpu


In [150]:
class FashionDNN(nn.Module):
    
    def __init__(self):
        super(FashionDNN,self).__init__()
        self.fc1 = nn.Linear(in_features=784, out_features=256)
        # 학습 중 랜덤하게 뉴런을 제거하면 특정 뉴런에 의존하는 것을 방지하고 더 일반적인 특징을 학습하게 됨
        self.drop = nn.Dropout(0,25)      # 0.25 → 25% 확률로 뉴런을 끈다(비활성화)
        self.fc2 = nn.Linear(in_features=256, out_features=128)
        self.fc3 = nn.Linear(in_features=128, out_features=10)
        
    def forward(self, input_data):
        # 전결합층은 1차원 형태의 입력만 받음 
        out = input_data.view(-1,784)
        out = F.relu(self.fc1(out))
        out = self.drop(out)
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        
        return out

In [151]:
from torchinfo import summary
m1= FashionDNN()
summary(m1, input_size=(1,28,28))

Layer (type:depth-idx)                   Output Shape              Param #
FashionDNN                               [1, 10]                   --
├─Linear: 1-1                            [1, 256]                  200,960
├─Dropout: 1-2                           [1, 256]                  --
├─Linear: 1-3                            [1, 128]                  32,896
├─Linear: 1-4                            [1, 10]                   1,290
Total params: 235,146
Trainable params: 235,146
Non-trainable params: 0
Total mult-adds (M): 0.24
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.94
Estimated Total Size (MB): 0.95

In [131]:
# DNN 모델에 이미지 테스트 (모델 만든 후 확인)
testIMG = trainDS.data[0:1].float()
pre = m1(testIMG).argmax()
pre, trainDS.targets[0]

(tensor(0), tensor(9))

In [ ]:
# 특징 추출 : 입력 이미지 주요 특징 추출한 특징맵 FeatureMap
#             이미지 채널 / 커널 수      출력 채널 / 커널 수      커널 크기       패딩크기        스트라이드
# 합성곱층  :            1                       1                  3             0               1
#                   커널 크기               스트라이드
# 풀 링 층  :           2                        2              다운샘플링 : 과적합 방지 (Max, Average)
# 플 랫 층  :  ---------------------------------------------------------------------------------------------
    
# 전결합학습 : 추출된 FeatureMap으로 학습 진행 
#             커널 수 * 높이 * 너비       출력 / 퍼셉트론 수 
# 은 닉 층  :       64 * 6 *6                  600
#                      600                     120
#                      120                     10

class FashionCNN(nn.Module):
    
    def __init__(self):
        super(FashionCNN, self).__init__()
        # 초기 레이어 : 큰 특성 맵을 유지해 다양한 패턴을 학습 -> Padding 사용(출력 크기 유지, 경계 정보 보존, 모델 설계 유연성)
        self.layer1 = nn.Sequential(nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
                                    nn.BatchNorm2d(32),   # 평균과 분산을 이용한 정규화 / out_channels와 동일 
                                    nn.ReLU(),            
                                    nn.MaxPool2d(kernel_size=2, stride=2))    # Thanos!!!!! : 보통 절반으로 줄이기 때문에 ks, strides 같은 경우 많음 
        # 후속 레이어 : 특성 맵의 크기를 축소하며 더 세밀한 정보를 추출 
        self.layer2 = nn.Sequential(nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
                                    nn.BatchNorm2d(64),
                                    nn.ReLU(),
                                    nn.MaxPool2d(2))      # 약식 표기 : stride를 따로 설정하지 않으면 자동으로 kernel_size와 동일한 값으로 설정됨 
        self.fc1 = nn.Linear(in_features=64*6*6, out_features=600)  # 풀링한 결과가 소수점일 시, 소수점 이하를 버림 
        self.drop = nn.Dropout2d(0.25)
        self.fc2 = nn.Linear(in_features=600, out_features=120)
        self.fc3 = nn.Linear(in_features=120, out_features=10)
        
        
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = self.fc3(out)
        
        return out
        

In [142]:
from torchinfo import summary
m1 = FashionCNN()

# 입력 크기를 올바르게 지정 : batch_size, channels, height, width
summary(m1, input_size=(1, 1, 28, 28)) 

Layer (type:depth-idx)                   Output Shape              Param #
FashionCNN                               [1, 10]                   --
├─Sequential: 1-1                        [1, 32, 14, 14]           --
│    └─Conv2d: 2-1                       [1, 32, 28, 28]           320
│    └─BatchNorm2d: 2-2                  [1, 32, 28, 28]           64
│    └─ReLU: 2-3                         [1, 32, 28, 28]           --
│    └─MaxPool2d: 2-4                    [1, 32, 14, 14]           --
├─Sequential: 1-2                        [1, 64, 6, 6]             --
│    └─Conv2d: 2-5                       [1, 64, 12, 12]           18,496
│    └─BatchNorm2d: 2-6                  [1, 64, 12, 12]           128
│    └─ReLU: 2-7                         [1, 64, 12, 12]           --
│    └─MaxPool2d: 2-8                    [1, 64, 6, 6]             --
├─Linear: 1-3                            [1, 600]                  1,383,000
├─Dropout2d: 1-4                         [1, 600]                  --
├─

In [143]:
# 모델 이미지 적용 후 체크
for img, target in testDS:
    print(img.shape, target)  # 예상: torch.Size([1, 28, 28])

    img = img.unsqueeze(0)  # 배치 차원 추가 → (1, 1, 28, 28)
    
    print(img.shape)  # 예상: torch.Size([1, 1, 28, 28])
    
    pre = m1(img)  # 이제 정상 동작
    print(pre, target)
    break

torch.Size([1, 28, 28]) 9
torch.Size([1, 1, 28, 28])
tensor([[-0.0659, -0.0661, -0.0505, -0.1157,  0.3675,  0.4871,  0.1729,  0.0051,
         -0.1397,  0.2033]], grad_fn=<AddmmBackward0>) 9
